# Dataset: https://www.kaggle.com/datasets/jangedoo/utkface-new

In [2]:
# unzipping the dataset
import zipfile
zip_ref = zipfile.ZipFile('/content/data.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [1]:
!pip install tensorflow==2.10.0
!pip install keras==2.10.0

In [2]:
import keras
import tensorflow as tf

print("Keras version:", keras.__version__)
print("TensorFlow version:", tf.__version__)


Keras version: 2.10.0
TensorFlow version: 2.10.0


In [2]:
import os
import cv2
import numpy as np
from keras.utils import Sequence
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

# Image dimensions
img_size = (100, 100)

# Custom Data Generator class
class DataGenerator(Sequence):
    def __init__(self, img_paths, labels, batch_size=32, img_size=(100, 100)):
        self.img_paths = img_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size

    def __len__(self):
        return int(np.ceil(len(self.img_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_paths = self.img_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = []
        ages = []
        genders = []
        for path, label in zip(batch_paths, batch_labels):
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.img_size)
            images.append(img)
            ages.append(label[0])
            genders.append(label[1])

        images = np.array(images) / 255.0
        ages = np.array(ages)
        genders = np.array(genders)

        return images, [ages, genders]

# Path to the dataset
path = "/content/data/UTKFace"

# Prepare image paths and labels
img_paths = [os.path.join(path, img) for img in os.listdir(path)]
labels = [(int(img.split("_")[0]), int(img.split("_")[1])) for img in os.listdir(path)]

# Split data into train and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(img_paths, labels, test_size=0.2, random_state=100)

# Create Data Generators
train_gen = DataGenerator(train_paths, train_labels, batch_size=32, img_size=img_size)
test_gen = DataGenerator(test_paths, test_labels, batch_size=32, img_size=img_size)

# Model architecture
input = Input(shape=(img_size[0], img_size[1], 3))
conv1 = Conv2D(64, (3, 3), activation="relu")(input)
pool1 = MaxPool2D((2, 2))(conv1)
conv2 = Conv2D(32, (3, 3), activation="relu")(pool1)
pool2 = MaxPool2D((2, 2))(conv2)
conv3 = Conv2D(16, (3, 3), activation="relu")(pool2)
pool3 = MaxPool2D((2, 2))(conv3)
flatten = Flatten()(pool3)

# Age prediction
age_l = Dense(32, activation="relu")(flatten)
age_l = Dense(16, activation="relu")(age_l)
age_l = Dense(1)(age_l)

# Gender prediction
gender_l = Dense(32, activation="relu")(flatten)
gender_l = Dense(16, activation="relu")(gender_l)
gender_l = Dropout(0.5)(gender_l)
gender_l = Dense(2, activation="softmax")(gender_l)

# Compile model
model = Model(inputs=input, outputs=[age_l, gender_l])
model.compile(optimizer="adam", loss=["mse", "sparse_categorical_crossentropy"], metrics=['mae', 'accuracy'])

# Callbacks
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True)
early_stop = EarlyStopping(monitor="val_loss", patience=5)

# Model training
save = model.fit(train_gen, validation_data=test_gen, epochs=50, callbacks=[checkpoint, early_stop])

# Save final model
model.save("final_model.h5")


Epoch 1/50
593/593 [==============================] - 477s 802ms/step - loss: 378.2160 - dense_2_loss: 377.5555 - dense_5_loss: 0.6602 - dense_2_mae: 14.7633 - dense_2_accuracy: 0.0483 - dense_5_mae: 0.5000 - dense_5_accuracy: 0.5878 - val_loss: 273.6297 - val_dense_2_loss: 273.0755 - val_dense_5_loss: 0.5543 - val_dense_2_mae: 12.0583 - val_dense_2_accuracy: 0.0426 - val_dense_5_mae: 0.5000 - val_dense_5_accuracy: 0.7189
Epoch 2/50
593/593 [==============================] - 452s 763ms/step - loss: 237.1796 - dense_2_loss: 236.6334 - dense_5_loss: 0.5461 - dense_2_mae: 11.5733 - dense_2_accuracy: 0.0465 - dense_5_mae: 0.5000 - dense_5_accuracy: 0.7361 - val_loss: 187.4272 - val_dense_2_loss: 186.9309 - val_dense_5_loss: 0.4962 - val_dense_2_mae: 10.1510 - val_dense_2_accuracy: 0.0382 - val_dense_5_mae: 0.5000 - val_dense_5_accuracy: 0.7482
Epoch 3/50
593/593 [==============================] - 459s 775ms/step - loss: 194.8064 - dense_2_loss: 194.3266 - dense_5_loss: 0.4799 - dense_2_mae